In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Import the data
data = pd.read_csv(r"mnist.csv")# Import the data

# Calling head will show the first 5 rows of the dataset
data.head()

# Work with numpy arrays and not pandas
data = np.array(data)
# Get the dimensions of the data. m is the amount of rows and n is the amount of columns
m, n = data.shape
np.random.shuffle(data)

# Split the data into training and testing data. .T is transpose, which is flipping the data, so each column is a row
# data_dev is the data we will use to test our model
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

# data_train is the data we will use to train our model
data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]

# Normalize pixel values to 0-1 range for better training
X_train = X_train / 255.0
X_dev = X_dev / 255.0

# Initialize parameters
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

# ReLU activation function. X if x > 0 and 0 if x <= 0
def ReLU(Z):
    # This will go through each element of the array and if it is greater than 0, it will keep it. If it is less than 0, it will make it 0
    return np.maximum(0, Z)

# Softmax activation function. This will take the exponential of each element in the array and then divide it by the sum of the exponential of each element in the array
def softmax(Z):
    # Numerically stable softmax
    A = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return A / np.sum(A, axis=0, keepdims=True)

# Defining forward propagation
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)  # Fixed: was ReLU(Z) instead of ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

# Defining one hot encoding, which is a way to represent categorical data as a vector of numbers
def one_hot(Y):
    # This will create a matrix of zeros with the same size as Y. Then it will go through each element of Y and make the corresponding element in the matrix 1
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    # This will go through each element of Y and make the corresponding element in the matrix 1
    one_hot_Y[np.arange(Y.size), Y] = 1
    # This will transpose or flip the matrix so that it is the same size as Y
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

# Defining ReLU derivative. This will return 1 if the element is greater than 0 and 0 if it is less than 0
def deriv_ReLU(Z):
    return Z > 0

# Defining backward propagation
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis=1, keepdims=True)  # Fixed: sum along axis 1, keep dims
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)  # Fixed: use deriv_ReLU function
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis=1, keepdims=True)  # Fixed: sum along axis 1, keep dims
    return dW1, db1, dW2, db2

# Defining update parameters
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

# Get predictions - returns the index of the highest value in the array
def get_predictions(A2):
    return np.argmax(A2, 0)

# Get accuracy
def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

# Defining gradient descent
def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        # Print the accuracy every 10 iterations
        if i % 10 == 0:
            # Calculate accuracy as the percentage of correct predictions out of total predictions
            accuracy = get_accuracy(get_predictions(A2), Y)
            print(f"Iteration: {i}")
            # Display accuracy as both decimal (0.0-1.0) and percentage (0%-100%) for clarity
            print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    return W1, b1, W2, b2

# Train the model with proper learning rate
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.5, 500)

# Test on validation set
Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X_dev)
dev_predictions = get_predictions(A2)
dev_accuracy = get_accuracy(dev_predictions, Y_dev)
print(f"Validation Accuracy: {dev_accuracy:.4f} ({dev_accuracy*100:.2f}%)")

# Visualization function to see predictions
def show_prediction(index):
    # Fashion-MNIST class names
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
    
    # Get the image and reshape it to 28x28
    current_image = X_dev[:, index].reshape((28, 28))
    
    # Make prediction
    prediction_input = X_dev[:, index, None]  # Add dimension for prediction
    Z1_pred, A1_pred, Z2_pred, A2_pred = forward_prop(W1, b1, W2, b2, prediction_input)
    prediction = get_predictions(A2_pred)[0]
    
    # Plot the image
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(current_image, cmap='gray')
    plt.title(f'Actual: {class_names[Y_dev[index]]}')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(current_image, cmap='gray')
    plt.title(f'Predicted: {class_names[prediction]}')
    plt.axis('off')
    
    # Show if prediction is correct
    correct = "✓" if prediction == Y_dev[index] else "✗"
    plt.suptitle(f'Prediction {correct}', fontsize=16, color='green' if correct == "✓" else 'red')
    plt.tight_layout()
    plt.show()
    
    print(f"Actual: {class_names[Y_dev[index]]}, Predicted: {class_names[prediction]}")

# Show a few examples
print("Showing Fashion-MNIST predictions:")
for i in range(10):
    show_prediction(i)